In [1]:
%pip install selenium
%pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
import time

In [3]:
#테스트할 인스타그램 계정 정보
username = '5steamtest'
password = 'testTest1!'

In [4]:
# Chrome 드라이버 초기화 및 설정
def init_driver():
    chrome_options = Options()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [5]:
# 인스타그램 로그인
def insta_login(driver, username, password):
    driver.get('https://www.instagram.com/accounts/login/')
    
    # 로그인 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'username')))
    username_input = driver.find_element(By.NAME, 'username')
    password_input = driver.find_element(By.NAME, 'password')
    
    # 로그인 정보 입력 및 로그인 버튼 클릭
    username_input.send_keys(username)
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    time.sleep(5)  # 페이지 로드 대기

In [6]:
# 사용자 프로필 페이지로 이동
def insta_search_user(driver, user_account):
    profile_url = f"https://www.instagram.com/{user_account}/"
    driver.get(profile_url)
    time.sleep(10)  # 프로필 로드 대기

In [7]:
# 최근 n개 게시물에서 좋아요를 한 사용자 목록 가져오기
def get_likes(driver, contents_num, username):
    #posts = driver.find_elements(By.XPATH, '//a[contains(@href, "/p/") or contains(@href, "/reels/")]')
    posts = driver.find_elements(By.XPATH, '//a[contains(@class, "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd")]')    
    post_links = []
    for post in posts[:contents_num]:
        link = post.get_attribute('href')
        if "/p/" in link or "/reel/" in link:
            link = link.replace(username+"/", "").replace("/reel/", "/p/")
            post_links.append(link)
    
    #post_links = [post.get_attribute('href').replace(username+"/", "").replace("/reel/", "/p/") for post in posts[:contents_num]]  # 최근 게시물 5개 링크 저장
    #for post in post_links:
    #    print(post)
    
    # 사용자 아이디 수집용 dict
    like_users = dict()

    if not post_links:
        print(f"게시글이 없습니다.")
        return 

    for index, post_url in enumerate(post_links, start=1):
        # 좋아요 페이지로 이동
        driver.get(f'{post_url}liked_by/')
        time.sleep(2)  # 로딩 후 대기
        try:
            last_height = driver.execute_script("return document.body.scrollHeight")
            
            while True:
                # 좋아요 사용자 리스트 가져오기
                users = driver.find_elements(By.XPATH, '//span[contains(@class, "_ap3a _aaco _aacw _aacx _aad7 _aade")]')
                for user in users:
                    like_users[user.text] = like_users.get(user.text, 0) + 1
                # 스크롤 내리기
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                
                # 스크롤 후 대기
                time.sleep(1)

                # 새로운 높이 계산
                new_height = driver.execute_script("return document.body.scrollHeight")
                
                # 마지막 높이와 비교하여 더 이상 스크롤할 수 없으면 종료
                if new_height == last_height:
                    break
                last_height = new_height

            # 좋아요 사용자 출력
            #print(f"Post {index} URL: {post_url} - Liked Users: {', '.join(list(like_users)[:len(like_users)])}")
            print(f"Post {index} URL: {post_url} - Liked Users: {', '.join(f'{key}: {value}' for key, value in like_users.items())}")         
        except Exception as NoLikedList:
            print(f"좋아요 사용자 리스트를 찾는 데 실패했습니다: {NoLikedList}")
    return like_users

### 메인 함수
TODO list 확인

In [8]:
def main():
    # 검색할 사용자 계정 입력
    # TODO: 프론트엔드와 연결해서 사용자 입력 받는 걸로 수정
    user_account = 'seoha1203' #좋아요+게시글+릴스 계정
    #user_account = 'rmqdid' #좋아요+게시글 계정
    #user_account = '5steamtest' #좋아요 없는 계정
    #user_account = '5steam0925' #게시글 없는 계정

    # 드라이버 초기화 및 로그인
    driver = init_driver()
    insta_login(driver, username, password)

    # 사용자 계정 검색
    insta_search_user(driver, user_account)
    # TODO: 검색할 계정에 있는 게시물 수 확인
    contents_num = 10 #해당 계정 게시물 수 몇 개 있는지에 따라 제한 필요
    
    # 최근 게시물에서 좋아요 및 댓글을 단 사용자 목록 가져오기  
    liked_users = get_likes(driver, contents_num, user_account)
    #for key, value in liked_users.items():
    #    print(f'{key}: {value}')
    # TODO: 시각화 할 때 user_account 는 걸러야함
    
    # 완료 후 드라이버 종료
    time.sleep(5)  # 확인용 대기
    driver.quit()  # 드라이버 종료

if __name__ == "__main__":
    main()

Post 1 URL: https://www.instagram.com/p/C9shtUsBD8t/ - Liked Users: ye.on._.06: 1, iam_ringkim: 1, _weekyear: 1, daye.__.n: 1, yunsrcd: 1, ieejyn: 1, zeonghye: 1, haeng_inthere: 1, kimchxxze: 1, seohyuuuuuun: 1, kmymyka: 1, r.hyej: 1, _dongni: 1, axxle.ah: 1, zwinhe: 1, mineral_ouo: 1, o0v0b: 1, cay_plo: 1, ssoo_jj: 1, am.3.00: 1
Post 2 URL: https://www.instagram.com/p/C7qsvF5JHta/ - Liked Users: ye.on._.06: 2, iam_ringkim: 1, _weekyear: 2, daye.__.n: 1, yunsrcd: 2, ieejyn: 2, zeonghye: 1, haeng_inthere: 1, kimchxxze: 1, seohyuuuuuun: 1, kmymyka: 2, r.hyej: 2, _dongni: 1, axxle.ah: 1, zwinhe: 2, mineral_ouo: 1, o0v0b: 2, cay_plo: 1, ssoo_jj: 1, am.3.00: 1, meuno._.o: 1, hye_rin811: 1, je_seong2: 1, ju_hee2_2: 1, seoha1203: 1, mellow_shhong: 1, yo_zheng21: 1, jixhyunie: 1, kk_imny: 1, haru_film_bp: 1, neejugram: 1, doublej_k: 1, ttkkkxx: 1, gain.hair: 1, merry_go_round_cake_: 1, dawngiraffe: 1, erin9751: 1, gyu_0212: 1, 2_taek: 1, y_driosn: 1, water_of_dawn: 1
Post 3 URL: https://www.in

----

In [ ]:
print()

아래는 각 함수 편하게 테스트 하려고 블럭 생성해 놓은 것

In [31]:
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

driver.get('https://www.instagram.com/accounts/login/')
    
# 로그인 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'username')))
username_input = driver.find_element(By.NAME, 'username')
password_input = driver.find_element(By.NAME, 'password')

# 로그인 정보 입력 및 로그인 버튼 클릭
username_input.send_keys(username)
password_input.send_keys(password)
password_input.send_keys(Keys.RETURN)
time.sleep(10)  # 페이지 로드 대기

user_account = 'rmqdid'
profile_url = f"https://www.instagram.com/{user_account}/"
driver.get(profile_url)
time.sleep(3)  # 프로필 로드 대기


In [170]:
posts = driver.find_elements(By.XPATH, '//a[contains(@href, "/p/")]')
post_links = [post.get_attribute('href') for post in posts[:5]]  # 최근 게시물 5개 링크 저장

# 사용자 아이디 수집용 dict
like_users = dict()
for index, post_url in enumerate(post_links, start=1):
    # 좋아요 페이지로 이동
    driver.get(f'{post_url}liked_by/')
    time.sleep(2)  # 로딩 후 대기
    try:
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            # 좋아요 사용자 리스트 가져오기
            users = driver.find_elements(By.XPATH, '//span[contains(@class, "_ap3a _aaco _aacw _aacx _aad7 _aade")]')
            for user in users:
                like_users[user.text] = like_users.get(user.text, 0) + 1
            # 스크롤 내리기
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # 스크롤 후 대기
            time.sleep(1)

            # 새로운 높이 계산
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            # 마지막 높이와 비교하여 더 이상 스크롤할 수 없으면 종료
            if new_height == last_height:
                break
            last_height = new_height

        # 좋아요 사용자 출력
        #print(f"Post {index} URL: {post_url} - Liked Users: {', '.join(list(like_users)[:len(like_users)])}")
        print(f"Post {index} URL: {post_url} - Liked Users: {', '.join(f'{key}: {value}' for key, value in like_users.items())}") 
    
    except Exception as e:
        print(f"좋아요 사용자 리스트를 찾는 데 실패했습니다: {e}")

Post 1 URL: https://www.instagram.com/p/DBAo18DPanG/ - Liked Users: chicken_mukgoshipda: 1, shyang0201: 1, tladbwls: 1, ibbi_nu: 1, sinj0916: 1, xeun.j: 1, yc_booktong: 1, hxj.o55: 1, yangyunseo02: 1, gno.ono: 1, o_gm__: 1, k_5hun: 1, chaseifucan: 1, woomin_0319: 1, m.xs0l: 1, this_wns: 1, _sxxhxxn_: 1, soeasy0104: 1, charis_to_mas: 1, jjungyui2: 1, song9l3: 1, aga_mandu: 1, fzrxxe: 1, zzl0g: 1, dhonne.z: 1, hxin.1e: 1, yxxj_h27: 1, civilian2121: 1, i_ji_you: 1, rla_minwoo: 1, naeunii_o.o: 1, jx_hegea.11: 1, j__da2: 1, riharibo: 1, zmin.v: 1, hebehfub: 1, seolraim1234: 1, h._.jin75: 1, iheeibee: 1, min_g001: 1, bo0u.10: 1, seo._y5: 1, topazung: 1, suwon.sexking: 1, shh._560: 1, j1._.h0on: 1, aze.n_12: 1, jeje_parkk: 1, tothenextround: 1, doicrossyourmind: 1, chng_oo: 1, kx._yn: 1, _eun_i__: 1, lovwe_.x: 1, jennyeoni: 1, intno.sohyang: 1, ex._.yun: 1, so_0.0b: 1, s._._.m1n: 1, hx.zyn0: 1, hayxji: 1, yjsyj_: 1, u0re.1: 1, 010_64x1_3xx9: 1, su._.sue_: 1, jinhee__0506: 1, somuch._.ing: 1, 

In [172]:
posts = driver.find_elements(By.XPATH, '//a[contains(@href, "/p/")]')
post_links = [post.get_attribute('href') for post in posts[:5]]  # 최근 게시물 5개 링크 저장

# 사용자 아이디 수집용 dict
comment_users = dict()
for index, post_url in enumerate(post_links, start=1):
    # 게시글 페이지로 이동
    driver.get(f'{post_url}')
    time.sleep(2)  # 로딩 후 대기
    try:
        # 댓글 사용자 리스트 가져오기
        comment_info = driver.find_elements(By.XPATH, '//span[contains(@class, "_x1lliihq _x1plvlek _xryxfnj _x1n2onr6 _x1ji0vk5 _x18bv5gf _x193iq5w _xeuugli _x1fj9vlw _x13faqbe _x1vvkbs _x1s928wv _xhkezso _x1gmr53x _x1cpjm7i _x1fgarty _x1943h6x _x1i0vuye _xvs91rp _xo1l8bm _x5n08af _x10wh9bi _x1wdrske _x8viiok _x18hxmgj")]')
        for info in comment_info:
            print(info)
        # 좋아요 사용자 출력
        #print(f"Post {index} URL: {post_url} - Comment Users: {', '.join(list(comment_users)[:len(comment_users)])}")
        #print(f"Post {index} URL: {post_url} - Comment Users: {', '.join(f'{key}: {value}' for key, value in comment_users.items())}") 
    
    except Exception as e:
        print(f"댓글 사용자 리스트를 찾는 데 실패했습니다: {e}")

In [32]:
# TODO: link에 user_account 이름 삽입
user_account = 'rmqdid'
driver.get(f'https://www.instagram.com/{user_account}/p/DAXT2j8zTno/')
#driver.get('https://www.instagram.com/p/DBGu-TQS33V/')
time.sleep(2)  # 로딩 후 대기

# 댓글 사용자 리스트 가져오기
comment_info = driver.find_elements(By.XPATH, '//span[contains(@class, "x1lliihq x1plvlek xryxfnj x1n2onr6 x1ji0vk5 x18bv5gf x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1i0vuye x133cpev x1xlr1w8 x5n08af x2b8uid x4zkp8e x41vudc x10wh9bi x1wdrske x8viiok x18hxmgj")]')
if len(comment_info) == 0:
    # TODO: while 댓글 더 로딩하는 버튼이 존재하면 계속 찾아서 클릭
    
    time.sleep(2)  # 버튼 클릭 후 대기
    
    comment_info = driver.find_elements(By.XPATH, '//span[contains(@class, "xt0psk2")]')
    # TODO: 계정 이름만 가져오는 처리 필요
    for info in comment_info:
        print(info.text)
else:
    print("No comment")


rmqdid
좋아요 2.5만개
rmqdid
ji.brightly
좋아요 1,597개
답글 달기
eunyul12
좋아요 257개
답글 달기
youcanevenbringyourbaby
좋아요 749개
답글 달기
z_pumpk1n
좋아요 3개
답글 달기
sidebyeon
좋아요 325개
답글 달기
yooonnii
좋아요 2개
답글 달기
drawing_pio
좋아요 150개
답글 달기
asukaji15
좋아요 10개
답글 달기
shaneeeee_hk
좋아요 3개
답글 달기
_yeonhoo26
좋아요 21개
답글 달기
iwannaextra8shots
좋아요 18개
답글 달기
nolimit.to_
좋아요 5개
답글 달기
8f85d8
좋아요 179개
답글 달기
sogm4_12
좋아요 1개
답글 달기
sd.0227
좋아요 1개
답글 달기
